In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\korea\anaconda3\envs\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install pandas

     ---------------------------------------- 11.0/11.0 MB 9.6 MB/s eta 0:00:00
     ------------------------------------- 498.1/498.1 kB 10.4 MB/s eta 0:00:00


In [3]:
!pip install sentence_transformers

# SentenceBERT 모델 로드

In [7]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["심리상담챗봇입니다.", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 619kB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 91.2kB/s]
Downloading: 100%|██████████| 4.86k/4.86k [00:00<00:00, 1.30MB/s]
Downloading: 100%|██████████| 931/931 [00:00<00:00, 460kB/s]
Downloading: 100%|██████████| 443M/443M [00:48<00:00, 9.17MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 26.5kB/s]
Downloading: 100%|██████████| 156/156 [00:00<00:00, 57.6kB/s]
Downloading: 100%|██████████| 495k/495k [00:01<00:00, 433kB/s]  
Downloading: 100%|██████████| 585/585 [00:00<00:00, 289kB/s]
Downloading: 100%|██████████| 248k/248k [00:00<00:00, 329kB/s]  
Downloading: 100%|██████████| 229/229 [00:00<00:00, 112kB/s]


[[ 0.02291415 -0.2434805   0.32847267 ...  0.29510644  0.21430862
   0.00397172]
 [-0.09361681 -0.18191518 -0.19230832 ... -0.03165777  0.30412534
  -0.2679363 ]]


# 데이터셋 로드

웰니스 대화 스크립트 데이터셋

https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100

In [8]:
df = pd.read_csv('wellness_dataset_original.csv')

df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


# 전처리

In [9]:
df = df.drop(columns=['Unnamed: 3'])

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [10]:
df = df[~df['챗봇'].isna()]

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [11]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [12]:
model.encode(df.loc[0, '유저'])

array([-4.80606616e-01, -2.94869423e-01,  4.37900215e-01, -6.40137851e-01,
        3.28670740e-02, -3.42647374e-01, -5.47481179e-02,  1.73056182e-02,
       -4.08221096e-01, -5.06034076e-01, -1.68733463e-01, -3.98676932e-01,
       -1.24776840e-01, -9.71538201e-02, -1.65286884e-01,  5.72612369e-03,
        6.13690987e-02, -1.91312388e-01,  2.53917605e-01, -5.85019708e-01,
       -2.84425557e-01, -2.32034907e-01, -3.27080548e-01,  6.72975630e-02,
       -1.62805827e-05, -4.72336292e-01, -3.60021859e-01,  2.91879863e-01,
       -6.63861156e-01, -3.10574383e-01,  5.79524994e-01, -3.11722606e-01,
        1.47696752e-02, -2.12172851e-01,  2.22057894e-01, -1.73828974e-01,
       -3.78458500e-01, -4.20398414e-01, -2.38218918e-01,  6.38703927e-02,
       -1.15304396e-01, -2.44563922e-01, -5.00228345e-01,  1.68355241e-01,
       -6.58360124e-01, -8.91941845e-01, -6.26956999e-01, -3.21965545e-01,
       -7.05358803e-01,  3.71447295e-01, -5.45803428e-01,  7.76295438e-02,
        1.09864593e-01,  

# 유저 대화내용 인코딩

In [13]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624..."


In [14]:
df.to_csv('wellness_dataset.csv', index=False)

# 간단한 챗봇

In [33]:
text = '사람 만나기가 무서워요'

embedding = model.encode(text)

In [34]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.tail()

,구분,유저,챗봇,embedding,distance
5196,치료이력/응급실,"그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...",응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?,"[-0.18907858, -0.43884447, 0.39459082, -0.2362...",0.232083
5197,치료이력/응급실,파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.,"[-0.5251449, -0.46516117, -0.26985374, -0.3506...",0.144253
5213,현재상태/증상악화,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.,"[-0.6814687, -0.11450246, 0.038397163, -0.2865...",0.400398
5214,현재상태/증상악화,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?,"[-0.18132992, -0.10980904, -0.21448663, -0.443...",0.299772
5215,현재상태/증상악화,그런데 증상이 나빠진 거 같아.,너무 심하시면 병원을 다시 가보는 건 어떨까요?,"[-0.53017116, -0.15990543, 0.49804136, -0.0031...",0.190525


In [35]:
answer = df.loc[df['distance'].idxmax()]

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['distance'])

구분 감정/두려움
유사한 질문 눈 뜨는게 두렵네요.
챗봇 답변 "우리가 유일하게 두려워해야 할 것은 두려움 그 자체다." 두려움이 당신을 잠식하도록 두지 마세요.
유사도 0.6564924716949463


In [26]:
!python chatbot.py

Traceback (most recent call last):
  File "chatbot.py", line 2, in <module>
    from streamlit_chat import message
ModuleNotFoundError: No module named 'streamlit_chat'


In [25]:
!pip install streamlit

  Using cached streamlit-1.14.0-py2.py3-none-any.whl (9.2 MB)
  Using cached rich-12.6.0-py3-none-any.whl (237 kB)
  Using cached validators-0.20.0.tar.gz (30 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pydeck-0.8.0-py2.py3-none-any.whl (4.7 MB)
     -------------------------------------- 904.4/904.4 kB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 20.0/20.0 MB 9.2 MB/s eta 0:00:00
  Using cached importlib_metadata-5.0.0-py3-none-any.whl (21 kB)
     ------------------------------------- 182.5/182.5 kB 11.5 MB/s eta 0:00:00
  Using cached semver-2.13.0-py2.py3-none-any.whl (12 kB)
  Using cached blinker-1.5-py2.py3-none-any.whl (12 kB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
     ---------------------------------------- 78.4/78.4 kB 4.5 MB/s eta 0:00:00
     ------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth 1.35.0 requires pyasn1-modules>=0.2.1, which is not installed.
google-auth 1.35.0 requires rsa<5,>=3.1.4; python_version >= "3.6", which is not installed.
google-auth-oauthlib 0.4.2 requires requests-oauthlib>=0.7.0, which is not installed.
google-auth 1.35.0 requires cachetools<5.0,>=2.0.0, but you have cachetools 5.2.0 which is incompatible.
